## QA Model Implementation
This project involves building a Question-Answering (QA) system that efficiently retrieves relevant documents from a vector store and generates context-aware answers using GPT-4. The system is designed to maintain conversation history for contextual responses and includes an option to end the conversation when a specific keyword ("quit") is detected.

### Key Steps in the Implementation:
1. SentenceTransformer Model:

>The SentenceTransformer model (paraphrase-MiniLM-L6-v2) is used to generate dense vector embeddings from text inputs. These embeddings capture the semantic meaning of sentences, making them suitable for similarity searches.

2. Dimensionality Reduction with PCA:

>Principal Component Analysis (PCA) is applied to the generated embeddings to reduce their dimensionality. This step is crucial for optimizing memory usage and improving the speed of similarity searches. The reduced embeddings are stored back into the vector store.

3. FAISS Indexing for Efficient Search:

>A FAISS (Facebook AI Similarity Search) index is built using the reduced embeddings. FAISS enables fast and scalable similarity searches, allowing the system to quickly retrieve the most relevant documents based on a query.

4. Query Processing and Search Function:

>The system includes a search function that encodes a user query into an embedding, applies PCA for dimensionality reduction, and searches the FAISS index to find the closest matching documents. The top k results are returned for further processing.

4. Answer Generation with GPT-4:

>The most relevant documents retrieved from the vector store are passed to the GPT-4 model, along with the user’s query, to generate a detailed and context-aware answer. The GPT-4 model synthesizes the information from the documents to provide a coherent response.

5. Maintaining Conversation Context:

>The system stores the conversation history, including all previous queries and answers, to maintain context throughout the interaction. This history is included in the prompt sent to GPT-4, enabling the model to generate responses that are consistent with the ongoing conversation.

6. Conversation Termination:

>The system includes a mechanism to end the conversation gracefully. If the user types "quit", the conversation loop is terminated, and the system exits.


## setup tools

In [3]:
# relevant libraies
import os
import openai
import faiss
import numpy as np
from dotenv import load_dotenv
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer


# Load environment variables from .env file
load_dotenv('../.env')

# Load OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OpenAI API key is not set in the environment variables.")
openai.api_key = api_key


import pickle

# Load vector store from disk
with open('../data/vector_stored.pkl', 'rb') as f:
    vector_store = pickle.load(f)



# Load your SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

embeddings_np = np.array(vector_store['embeddings'])
pca = PCA(n_components=100)  # Adjust the number of components as needed
embeddings_reduced = pca.fit_transform(embeddings_np)
vector_store['embeddings_reduced'] = embeddings_reduced

d = embeddings_reduced.shape[1]  # Dimensionality of the reduced embeddings
index = faiss.IndexFlatL2(d)  # L2 distance for exact search
index.add(embeddings_reduced)  # Add the reduced embeddings to the index


class QAChatAgent:
    def __init__(self, model, pca, index, vector_store, top_k=5):
        self.model = model
        self.pca = pca
        self.index = index
        self.vector_store = vector_store
        self.top_k = top_k
        self.conversation_history = []  # To store the conversation history

    def add_to_history(self, query, answer):
        self.conversation_history.append({"query": query, "answer": answer})

    def generate_answer_gpt4(self, relevant_documents, question):
        context = "\n\n".join([f"User: {entry['query']}\nAssistant: {entry['answer']}" for entry in self.conversation_history])
        context += "\n\n" + "\n\n".join(relevant_documents)
        prompt = f"Based on the following documents and conversation history, answer the question:\n\n{context}\n\nQuestion: {question}\nAnswer:"

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=350,
            temperature=0.0
        )
        
        return response.choices[0].message['content'].strip()

    def search_vector_store(self, query):
        query_embedding = self.model.encode(query)
        query_embedding_reduced = self.pca.transform([query_embedding]).astype('float32')
        D, I = self.index.search(query_embedding_reduced, self.top_k)
        results = [(self.vector_store['documents'][i], self.vector_store['metadatas'][i], D[0][n]) for n, i in enumerate(I[0])]
        return results

    def process_query(self, query):
        if query.lower().strip() == "quit":
            print("Ending conversation. Goodbye!")
            return None
        
        results = self.search_vector_store(query)
        relevant_documents = [doc for doc, _, _ in results]
        answer = self.generate_answer_gpt4(relevant_documents, query)
        
        # Add the query and answer to the conversation history
        self.add_to_history(query, answer)
        
        return answer
    

# Usage
qa_agent = QAChatAgent(model, pca, index, vector_store)

# Simulate conversation loop
while True:
    user_query = input("You: ")
    response = qa_agent.process_query(user_query)
    
    if response is None:
        break  # Exit the loop if "quit" is typed
    
    print(f"rgu_bot: {response}")

    


### load openAI key

In [4]:
# Load environment variables from .env file
load_dotenv('../.env')

# Load OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OpenAI API key is not set in the environment variables.")
openai.api_key = api_key


### load the website content

In [5]:
import pickle

# Load vector store from disk
with open('../data/vector_stored.pkl', 'rb') as f:
    vector_stored = pickle.load(f)

vector_store = vector_stored

In [6]:
# Load your SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

embeddings_np = np.array(vector_store['embeddings'])
pca = PCA(n_components=100)  # Adjust the number of components as needed
embeddings_reduced = pca.fit_transform(embeddings_np)
vector_store['embeddings_reduced'] = embeddings_reduced

d = embeddings_reduced.shape[1]  # Dimensionality of the reduced embeddings
index = faiss.IndexFlatL2(d)  # L2 distance for exact search
index.add(embeddings_reduced)  # Add the reduced embeddings to the index

In [7]:
class QAChatAgent:
    def __init__(self, model, pca, index, vector_store, top_k=5):
        self.model = model
        self.pca = pca
        self.index = index
        self.vector_store = vector_store
        self.top_k = top_k
        self.conversation_history = []  # To store the conversation history

    def add_to_history(self, query, answer):
        self.conversation_history.append({"query": query, "answer": answer})

    def generate_answer_gpt4(self, relevant_documents, question):
        context = "\n\n".join([f"User: {entry['query']}\nAssistant: {entry['answer']}" for entry in self.conversation_history])
        context += "\n\n" + "\n\n".join(relevant_documents)
        prompt = f"Based on the following documents and conversation history, answer the question:\n\n{context}\n\nQuestion: {question}\nAnswer:"

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=350,
            temperature=0.0
        )
        
        return response.choices[0].message['content'].strip()

    def search_vector_store(self, query):
        query_embedding = self.model.encode(query)
        query_embedding_reduced = self.pca.transform([query_embedding]).astype('float32')
        D, I = self.index.search(query_embedding_reduced, self.top_k)
        results = [(self.vector_store['documents'][i], self.vector_store['metadatas'][i], D[0][n]) for n, i in enumerate(I[0])]
        return results

    def process_query(self, query):
        if query.lower().strip() == "quit":
            print("Ending conversation. Goodbye!")
            return None
        
        results = self.search_vector_store(query)
        relevant_documents = [doc for doc, _, _ in results]
        answer = self.generate_answer_gpt4(relevant_documents, query)
        
        # Add the query and answer to the conversation history
        self.add_to_history(query, answer)
        
        return answer

In [ ]:
# Usage
qa_agent = QAChatAgent(model, pca, index, vector_store)

# Simulate conversation loop
while True:
    user_query = input("You: ")
    response = qa_agent.process_query(user_query)
    
    if response is None:
        break  # Exit the loop if "quit" is typed
    
    print(f"rgu_bot: {response}")


You:  hi


rgu_bot: Hello! How can I assist you today?


You:  who are you


rgu_bot: I am an AI assistant here to help you with any questions or tasks you have. How can I assist you today?


> <b>This implementation combines efficient vector search techniques with the powerful natural language generation capabilities of GPT-4, resulting in a robust and responsive QA system capable of handling complex user queries with contextually relevant answers.</b>